[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Tiger-Quant/demos2025/blob/master/sharpe_ratio_challenge.ipynb)

## Sharpe Ratio Challenge (Sept. 16)


Use this starter code to now calculate the annualized Sharpe Ratio for both SPY and Apple (AAPL) using data from the last two years. Remember, sharpe ratio = (the average return minus the risk-free rate) / (standard deviation of returns).

The risk-free rate is the return of a theoretically "zero-risk" investment. Using a constant like 4% is a common simplification. A more precise method, and the one we'll use, is to subtract the **actual historical risk-free rate** for each day from the stock's daily return. This gives us the true *excess return* for each specific day over the past two years. We'll use the 3-Month Treasury Bill rate as our proxy for the risk-free rate.

Run the cells in order below to see how to find the annualized Sharpe ratio for AAPL and SPY. 

In [1]:
# ------------------------------------------------------------------
# Setup Cell: Run this first to install required libraries.
# ------------------------------------------------------------------
# The '!' command runs a shell command. We use it to call pip.
# The '-q' flag makes the output "quiet" to keep the notebook clean.

!pip install -q yfinance pandas matplotlib-venn

print("✅ Setup complete. You can now run the rest of the notebook.")

✅ Setup complete. You can now run the rest of the notebook.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [4]:
# Define our assets and the date range

tickers = ['AAPL', 'SPY', '^IRX']  # Apple, S&P 500 ETF, 3-Month Treasury Bill
end_date = datetime.today()
start_date = end_date - timedelta(days=2*365)

data = yf.download(tickers, start=start_date, end=end_date)
data

C:\Users\Evan Schreiner\AppData\Local\Temp\ipykernel_13432\3275978392.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date)
[*********************100%***********************]  3 of 3 completed


Price            Close                           High                     \
Ticker            AAPL         SPY   ^IRX        AAPL         SPY   ^IRX   
Date                                                                       
2023-09-25  174.372025  421.340698  5.313  175.253391  421.379669  5.315   
2023-09-26  170.292023  415.150665  5.325  173.500586  418.991406  5.325   
2023-09-27  168.776840  415.316345  5.330  171.361524  416.895557  5.333   
2023-09-28  169.034302  417.724152  5.308  170.361300  419.410578  5.333   
2023-09-29  169.549255  416.710327  5.300  171.391214  420.970227  5.305   
...                ...         ...    ...         ...         ...    ...   
2025-09-18  237.880005  660.429016  3.880  241.199997  663.051750  3.880   
2025-09-19  245.500000  663.700012  3.878  246.300003  664.549988  3.878   
2025-09-22  256.079987  666.840027  3.848  256.640015  667.289978  3.875   
2025-09-23  254.429993  663.210022  3.845  257.339996  667.340027  3.848   
2025-09-24         NaN         NaN  3.858         NaN         NaN  3.858   

Price              Low                           Open                     \
Ticker            AAPL         SPY   ^IRX        AAPL         SPY   ^IRX   
Date                                                                       
2023-09-25  172.460738  417.919117  5.303  172.510256  418.357792  5.313   
2023-09-26  169.994930  414.312316  5.313  173.124282  418.279786  5.323   
2023-09-27  167.410236  411.651093  5.310  170.945600  416.330152  5.325   
2023-09-28  165.994074  414.166113  5.308  167.697391  414.760761  5.333   
2023-09-29  168.687684  415.179874  5.288  170.351396  420.794769  5.295   
...                ...         ...    ...         ...         ...    ...   
2025-09-18  236.649994  658.444528  3.860  239.970001  660.060044  3.863   
2025-09-19  240.210007  660.369995  3.878  241.229996  662.330017  3.878   
2025-09-22  248.119995  662.169983  3.840  248.300003  662.200012  3.873   
2025-09-23  253.580002  661.979980  3.843  255.880005  666.719971  3.845   
2025-09-24         NaN         NaN  3.848         NaN         NaN  3.850   

Price            Volume                    
Ticker             AAPL          SPY ^IRX  
Date                                       
2023-09-25   46172700.0   70874500.0    0  
2023-09-26   64588900.0   96168400.0    0  
2023-09-27   66921800.0  104705800.0    0  
2023-09-28   56294400.0   92258300.0    0  
2023-09-29   51861100.0  115111300.0    0  
...                 ...          ...  ...  
2025-09-18   44249600.0   90459200.0    0  
2025-09-19  163741300.0   97945600.0    0  
2025-09-22  105517400.0   69452200.0    0  
2025-09-23   60275200.0   81708900.0    0  
2025-09-24          NaN          NaN    0  

[502 rows x 15 columns]

**Clean up the data...**

Notice the extra step for the risk-free rate `^IRX`. Since it's an annualized percentage, we divide it by 100 to make it a decimal and then by 252 (the approximate number of trading days in a year) to get the daily rate.

In [6]:
# Keep only the 'Adj Close' price and drop any missing values
prices = data['Close'].dropna()

# Convert the risk free rate to a daily deciaml value
prices['^IRX'] = prices['^IRX'] / 100 / 252

prices

Ticker,AAPL,SPY,^IRX
Date,,,
2023-09-25,174.372025,421.340698,0.000211
2023-09-26,170.292023,415.150665,0.000211
2023-09-27,168.776840,415.316345,0.000212
2023-09-28,169.034302,417.724152,0.000211
2023-09-29,169.549255,416.710327,0.000210
...,...,...,...
2025-09-17,238.990005,657.357544,0.000153
2025-09-18,237.880005,660.429016,0.000154
2025-09-19,245.500000,663.700012,0.000154


**Calculate Excess Returns**

Now that we have the daily prices, the next step toward the Sharpe Ratio is calculating the daily returns for SPY and AAPL.

In [8]:
# Calculate daily returns for the stocks
daily_returns = prices[['AAPL', 'SPY']].pct_change()

# Calculate the daily excess returns
excess_returns = daily_returns.subtract(prices['^IRX'], axis=0)

# Drop any missing values and preview
excess_returns = excess_returns.dropna()
excess_returns

Ticker,AAPL,SPY
Date,,
2023-09-26,-0.023610,-0.014903
2023-09-27,-0.009109,0.000188
2023-09-28,0.001315,0.005587
2023-09-29,0.002836,-0.002637
2023-10-02,0.014625,-0.000609
...,...,...
2025-09-17,0.003374,-0.001396
2025-09-18,-0.004799,0.004518
2025-09-19,0.031879,0.004799


**Get the Sharpe Ratio**

Let's revisit the formula: The numerator (R_p−R_f) is the average daily excess return, and the denominator (σ_p) is the standard deviation of the daily excess returns.

We now have the daily excess returns. Let's get the numerator and denominator.

In [9]:
avg_excess_return = excess_returns.mean()   # Numerator
std_excess_return = excess_returns.std()    # Denominator

daily_sharpe_ratio = avg_excess_return / std_excess_return
print(f'Daily Sharpe Ratio: {daily_sharpe_ratio}')

Daily Sharpe Ratio: Ticker
AAPL    0.041026
SPY     0.074952
dtype: float64


**Annualize the Sharpe Ratio**

This gives us a daily Sharpe Ratio, which isn't very intuitive. To make it meaningful, we need to convert it into a yearly figure. The final step is to **annualize** the ratio. We do this to compare investments on a standard, one-year timeframe. Any idea how we might convert a daily statistical measure into an annual one? Hint: It involves the number of trading days in a year (usually 252).

While the average return scales directly with time (so we'd multiply that by 252), the risk (standard deviation) scales by the **square root of time**. Because of this, to annualize the entire ratio, we multiply it by the square root of 252.

In [12]:
annualized_sharpe_ratio = daily_sharpe_ratio* np.sqrt(252)
print("Annualized Sharpe Ratio (Last 2 Years):")
print(annualized_sharpe_ratio)

Annualized Sharpe Ratio (Last 2 Years):
Ticker
AAPL    0.651275
SPY     1.189828
dtype: float64


Based on these results, SPY offered a better return for the amount of risk taken over the past two years.

## Recap: Calculating the Annualized Sharpe Ratio

The process can be broken down into four main steps.


1. **Gather Daily Data** 📥

First, we collected the daily historical data for our chosen assets (AAPL and SPY) and a proxy for the risk-free rate (the 13-week T-Bill, `IRX`).
* **Stocks**: We use the 'Close' price.
* **Risk-Free Rate**: The T-Bill rate is typically an annualized percentage, so we convert it to a daily decimal by dividing by 100 and then by 252.


2. **Calculate Daily Excess Returns** 📈

Next, we calculated the daily returns for our stocks using the percent change from one day to the next (`.pct_change()`). We then subtracted the daily risk-free rate from these returns to find the **excess return**.

$$ \text{Excess Return} = \text{Asset’s Daily Return} − \text{Daily Risk Free Rate} $$


3. **Calculate the Daily Sharpe Ratio** ➗

Using the series of daily excess returns, we calculated the daily Sharpe Ratio. This involves two key stats:

* **Numerator:** The average of the daily excess returns (.mean()).
**Denominator:** The standard deviation of the daily excess returns (.std()).

$$ \text{Daily Sharpe Ratio} = \frac{\text{Average Daily Excess Return}}{\text{Standard Deviation of Daily Excess Returns}} $$
​
 
4. **Annualize the Sharpe Ratio** 🌱

The final and most important step is to convert the daily ratio into a more meaningful annualized figure.

The key principle here is that **returns scale linearly with time, but risk (standard deviation) scales with the square root of time.**

* If you double the time period, you expect to double your return.
* However, you only increase your risk by the square root of 2 (about 1.41x).

Because the Sharpe Ratio is a measure of return divided by risk, its scaling factor is the time period divided by the square root of the time period. For annualizing daily data, where the time period `T` is 252 trading days:

$$ \frac{\text{Time}}{\sqrt{\text{Time}}} = \frac{T}{\sqrt{T}} = \sqrt{T} $$


Therefore, to convert our daily Sharpe Ratio to an annual one, we multiply it by the **square root of 252**.

$$ \text{Annualized Sharpe Ratio} = \text{Daily Sharpe Ratio} × \sqrt{252} $$

========

*As a suggestion for best practice, while the Sharpe Ratio is excellent, it's often used alongside other metrics like the Sortino Ratio (which only considers downside risk) or Maximum Drawdown (which measures the largest peak-to-trough decline) for a more complete performance analysis.*